# 🎬 Movie Revenue Prediction: Full Pipeline & Comparison

Notebook này sẽ chạy quy trình **End-to-End** từ A-Z và so sánh trực tiếp hiệu quả giữa **Phiên bản cũ (V1)** và **Phiên bản nâng cao (V2)**.

### 📌 Quy trình:
1.  **Data Loading**: Tải dữ liệu phim (2010-2024).
2.  **Experiment 1 (Baseline)**: Chạy Preprocessing V1 (Old strategy).
3.  **Experiment 2 (Advanced)**: Chạy Preprocessing V2 (New strategy: KNN, RobustScaler, BGE Embeddings).
4.  **Comparison**: So sánh R2 Score & MAE.
5.  **Final Model & Data**: Lưu lại mô hình và dữ liệu (Train/Test) từ V2.

In [18]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore') # Tắt warning cho gọn

# Setup Path
current_dir = Path(os.getcwd())
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
else:
    project_root = current_dir

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import Modules
from src.data_loader import TMDbDataLoader
from src.preprocessing import DataPreprocessor        # V1
from src.preprocessing_v2 import DataPreprocessorV2   # V2 (New)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import xgboost as xgb
import joblib

print(f"Project Root: {project_root}")

Project Root: d:\Movie_Revenue_Prediction_v2


## 1. Load Data

In [19]:
config_path = project_root / "configs" / "config.yaml"
raw_data_path = project_root / "data" / "raw" / "movies_2020_2024.csv"

loader = TMDbDataLoader(config_path=str(config_path))

if not raw_data_path.exists():
    print("Fetching data from API...")
    loader.fetch_data()
    loader.save_data(str(raw_data_path))
else:
    print("Loading existing data...")

df_raw = loader.load_data(str(raw_data_path))
print(f"Data Shape: {df_raw.shape}")

2025-12-07 21:36:53,249 - src.data_loader - INFO - TMDbDataLoader đã được khởi tạo thành công


Loading existing data...


2025-12-07 21:36:53,456 - src.data_loader - INFO - Loaded 2708 rows từ d:\Movie_Revenue_Prediction_v2\data\raw\movies_2020_2024.csv


Data Shape: (2708, 14)


## 2. Setting Up Experiments (V1 vs V2)

In [20]:
def run_experiment(name, preprocessor, model, df):
    print(f"\n{'='*20} RUNNING: {name} {'='*20}")
    
    # 1. Preprocessing
    print(" Running fit_transform...")
    X, y = preprocessor.fit_transform(df)
    print(f"Features Shape: {X.shape}")
    
    # 2. Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 3. Train
    print("Training Model...")
    model.fit(X_train, y_train)
    
    # 4. Evaluate
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"{name} Results:")
    print(f"   R2 Score: {r2:.4f}")
    print(f"   MAE:      ${mae:,.2f}")
    
    return {
        "Experiment": name,
        "R2": r2,
        "MAE": mae,
        "Features": X.shape[1],
        "Test_Samples": len(y_test),
        "Model": model,
        "Preprocessor": preprocessor,
        "Data": (X_train, X_test, y_train, y_test) # Save data for later
    }

## 3. Run V1 (Baseline)

In [21]:
# V1: Basic Preprocessing + RandomForest
prep_v1 = DataPreprocessor(config_path=str(config_path))
model_v1 = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

res_v1 = run_experiment("V1 (Basic)", prep_v1, model_v1, df_raw)

2025-12-07 21:36:53,561 - src.preprocessing - INFO - DataPreprocessor đã được khởi tạo thành công (target_col=revenue, scaler=StandardScaler)
2025-12-07 21:36:53,574 - src.preprocessing - INFO - Bắt đầu fit preprocessing pipeline...
2025-12-07 21:36:53,612 - src.preprocessing - INFO - Đã xử lý missing values
2025-12-07 21:36:53,646 - src.preprocessing - INFO - Đã loại bỏ 344 outliers (12.70%)



==================== RUNNING: V1 (Basic) ====================
 Running fit_transform...


2025-12-07 21:36:53,707 - src.preprocessing - INFO - Đã tạo 5 date features
2025-12-07 21:36:53,750 - src.preprocessing - INFO - Đã encode 19 genres
2025-12-07 21:36:54,118 - src.preprocessing - INFO - Đã tạo 30 TF-IDF features từ overview
2025-12-07 21:36:54,130 - src.preprocessing - INFO - Đã tạo derived features
2025-12-07 21:36:54,147 - src.preprocessing - INFO - Đã fit preprocessor với 65 features
2025-12-07 21:36:54,148 - src.preprocessing - INFO - Bắt đầu transform data...
2025-12-07 21:36:54,152 - src.preprocessing - INFO - Đã xử lý missing values
2025-12-07 21:36:54,167 - src.preprocessing - INFO - Đã loại bỏ 344 outliers (12.70%)
2025-12-07 21:36:54,183 - src.preprocessing - INFO - Đã tạo 5 date features
2025-12-07 21:36:54,193 - src.preprocessing - INFO - Đã encode 19 genres
2025-12-07 21:36:54,343 - src.preprocessing - INFO - Đã tạo 30 TF-IDF features từ overview
2025-12-07 21:36:54,347 - src.preprocessing - INFO - Đã tạo derived features
2025-12-07 21:36:54,356 - src.prepr

Features Shape: (2364, 65)
Training Model...
V1 (Basic) Results:
   R2 Score: 0.5896
   MAE:      $28,956,918.54


## 4. Run V2 (Advanced with Embeddings)

In [22]:
# V2: Advanced Preprocessing (KNN, RobustScaler, BGE) + XGBoost (Mạnh hơn RF)
prep_v2 = DataPreprocessorV2(config_path=str(config_path))
# Sử dụng XGBoost cho V2 vì nó thường handle features tốt hơn RF
model_v2 = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, n_jobs=-1, random_state=42)

res_v2 = run_experiment("V2 (Advanced)", prep_v2, model_v2, df_raw)

2025-12-07 21:36:56,194 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-12-07 21:36:56,196 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2025-12-07 21:37:01,957 - src.preprocessing_v2 - INFO - Using Embedding Model: BAAI/bge-small-en-v1.5
2025-12-07 21:37:01,958 - src.preprocessing_v2 - INFO - DataPreprocessorV2 (Advanced) đã được khởi tạo.
2025-12-07 21:37:01,965 - src.preprocessing_v2 - INFO - Bắt đầu fit DataPreprocessorV2 (Advanced)...
2025-12-07 21:37:02,023 - src.preprocessing_v2 - INFO - Encoding overview with BGE Embeddings...



==================== RUNNING: V2 (Advanced) ====================
 Running fit_transform...


2025-12-07 21:39:33,331 - src.preprocessing_v2 - INFO - Đã tạo 384 features từ BGE Embeddings
2025-12-07 21:39:34,526 - src.preprocessing_v2 - INFO - Đã fit V2 với 419 features.
2025-12-07 21:39:35,292 - src.preprocessing_v2 - INFO - Encoding overview with BGE Embeddings...
2025-12-07 21:41:28,402 - src.preprocessing_v2 - INFO - Đã tạo 384 features từ BGE Embeddings


Features Shape: (2708, 419)
Training Model...
V2 (Advanced) Results:
   R2 Score: 0.7671
   MAE:      $53,511,084.00


## 5. Final Comparison & Save Data

In [23]:
# Tổng hợp kết quả
df_results = pd.DataFrame([res_v1, res_v2])
cols = ['Experiment', 'R2', 'MAE', 'Features']
print("\nLEADERBOARD")
print(df_results[cols])

print("\nSaving Best Model & Data (V2)...")
models_dir = project_root / "models"
processed_dir = project_root / "data" / "processed"
models_dir.mkdir(parents=True, exist_ok=True)
processed_dir.mkdir(parents=True, exist_ok=True)

# 1. Lưu Model & Preprocessor
joblib.dump(res_v2['Model'], models_dir / "best_model.pkl")
res_v2['Preprocessor'].save_preprocessor(str(models_dir / "preprocessor.pkl"))

# 2. Lưu Processed Data (Train/Test) của V2
X_train_v2, X_test_v2, y_train_v2, y_test_v2 = res_v2['Data']

feature_names = res_v2['Preprocessor'].get_feature_names()
# Lưu X_train, X_test
pd.DataFrame(X_train_v2, columns=feature_names).to_csv(processed_dir / "X_train.csv", index=False)
pd.DataFrame(X_test_v2, columns=feature_names).to_csv(processed_dir / "X_test.csv", index=False)
# Lưu y_train, y_test
pd.DataFrame(y_train_v2, columns=['revenue']).to_csv(processed_dir / "y_train.csv", index=False)
pd.DataFrame(y_test_v2, columns=['revenue']).to_csv(processed_dir / "y_test.csv", index=False)

print(f"Done! Data saved to '{processed_dir}'")


LEADERBOARD
      Experiment        R2           MAE  Features
0     V1 (Basic)  0.589595  2.895692e+07        65
1  V2 (Advanced)  0.767092  5.351108e+07       419

Saving Best Model & Data (V2)...


2025-12-07 21:41:52,102 - src.preprocessing_v2 - INFO - Đã lưu preprocessor vào: d:\Movie_Revenue_Prediction_v2\models\preprocessor.pkl


Done! Data saved to 'd:\Movie_Revenue_Prediction_v2\data\processed'
